In [3]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,065 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,809 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.lau

In [4]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

# Creamos el Spark Context

In [6]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
DATA_PATH = "/content/drive/MyDrive/CienciaDeDatos/TP1/data/"

sqlContext = SQLContext(sc)

rdd_order_items = (
    sqlContext.read.csv(DATA_PATH + "order_items.csv", header=True, inferSchema=True)
    .select("order_id", 'product_id', "quantity")
    .rdd
    )


/usr/local/lib/python3.12/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


# 6) Obtener la cantidad de órdenes que no hayan comprado ninguno de los 10 productos más vendidos.

In [27]:
K_TOP = 10

top_k_most_sold_products = (
    rdd_order_items
    .filter(lambda r: r["quantity"] and r["product_id"])
    .map(lambda r: (int(r["product_id"]), int(r["quantity"])))
    .reduceByKey(lambda a, b: a + b)
    .sortBy(lambda x: x[1], ascending=False)
    .take(K_TOP)
)

print("---------------------- TOP K Products -------------------")
print(top_k_most_sold_products.__str__())
print("---------------------------------------------------------")
set_top_k = set(map(lambda x: x[0], top_k_most_sold_products))

orders_with_top_k = (
    rdd_order_items
    .filter(lambda r: r["order_id"] and r["product_id"] and r["quantity"])
    .map(lambda r: (int(r["order_id"]), int(r["product_id"]) in top_ids))
    .reduceByKey(lambda a, b: a or b)
)

amount_orders_top_k = (
    orders_with_top_k
    .map(lambda kv: (kv[1], 1))
    .reduceByKey(lambda a, b: a + b)
    .collectAsMap()
)

without_top = amount_orders_top_k.get(False, 0)
with_top = amount_orders_top_k.get(True, 0)
total_seen = with_top + without_top

print("n")
print("----------------------- Resultados ----------------------")
print(f"Órdenes consideradas: {total_seen}")
print(f"Con top-{K_TOP}: {with_top}")
print(f"Sin top-{K_TOP}: {without_top}")


---------------------- TOP K Products -------------------
[(992501, 69), (945329, 67), (999049, 65), (902689, 65), (906628, 64), (936039, 64), (984570, 63), (953564, 63), (951114, 63), (941797, 63)]
---------------------------------------------------------
n
----------------------- Resultados ----------------------
Órdenes consideradas: 95585
Con top-10: 88
Sin top-10: 95497


# Conclusiones

De 95.585 órdenes consideradas, solo 88 incluyen alguno de los 10 productos más vendidos y 95.497 no, a primera vista estos números parecen incoherentes. Sin embargo, al considerar que el dataset está totalmente randomizado y que se cuenta con una enorme cantidad de productos, se entiende que la aparición de productos en cada orden sigue una distribución normal y por lo tanto el valor de un 99% de ordenes sin esos productos cobra más sentido. De todas formas llama la atención el valor tan elevado de órdenes sin el top 10, y se cree que en cualquier dataset con información de la vida real, el valor será notablemente más bajo, ya que en cualquier situación normal, los productos más vendidos de un sistema deben aparecer muchas más veces y conformar a mayor cantidad de las órdenes.